## FCN Model - 2015 NY datasets Deep <br>

### Date : 2015-01-01 ~ 2015-03-01 
### training : 40 day (20% val ) / 20 day 
#### Summary : 

----------------------



In [1]:

import pandas as pd
import numpy as np
import tensorflow as tf

import random as rn
from utils import *
import os
os.environ['PYTHONHASHSEED'] = '0'

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
rn.seed(RANDOM_SEED)
session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

import keras
import keras.backend as K

tf.set_random_seed(RANDOM_SEED)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

from keras import layers
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, AveragePooling2D, UpSampling2D, Flatten, Reshape, Dropout, Conv2DTranspose
from keras.layers import Concatenate, BatchNormalization, Add
from keras.models import Model, Sequential
from keras.layers import InputLayer
from keras.utils.training_utils import multi_gpu_model


###################
## Hyperparam  - 맞게 수정하거나 Arg로 받아 온다. (Data Folder / Model Name / Model Ver ) 추가 
###################


LRATE = 1e-3
BATCH_SIZE = 128
EPOCHS = 2000
SCALE = 'log' # or 'max' or 'log'
NGPU = 2

## Folder / File Name
DATA_FOLDER = '/data/public/rw/prj-mobility/datasets/NY_data/2ch/'

## Baseline Model Folder 
BASE_MODEL_NAME = 'MODEL_V1_NYC'
MODEL_VER = '0827_NY_001_2ch_test_base_1'


##################################################
OUTPUT_FOLDER = '../output_file/'
TF_FOLDER = '../tfgraph/'
MODEL_SAVE_FOLDER = '../saved_model/'

MODEL_NAME_ = BASE_MODEL_NAME+'_'+MODEL_VER
OUTPUT_FOLDER = OUTPUT_FOLDER+BASE_MODEL_NAME+'/'
TF_FOLDER = TF_FOLDER+BASE_MODEL_NAME+'/'+MODEL_VER
MODEL_SAVE_FOLDER = MODEL_SAVE_FOLDER+BASE_MODEL_NAME+'/'

print ('Model Name :',MODEL_NAME_)

if os.path.isdir(OUTPUT_FOLDER) == False:
    os.makedirs(OUTPUT_FOLDER)
    print ('Output Folder :',OUTPUT_FOLDER, 'created')
else :
    print ('Output Folder :',OUTPUT_FOLDER)

if os.path.isdir(TF_FOLDER) == False:
    os.makedirs(TF_FOLDER)
    print ('Tfgraph Folder :',TF_FOLDER, 'created')
else :
    print ('Tfgraph Folder :',TF_FOLDER)
    
if os.path.isdir(MODEL_SAVE_FOLDER) == False:
    os.makedirs(MODEL_SAVE_FOLDER)
    print ('Save Folder :',MODEL_SAVE_FOLDER, 'created')
else :
    print ('Save Folder :',MODEL_SAVE_FOLDER)




Using TensorFlow backend.


Model Name : MODEL_V1_NYC_0827_NY_001_2ch_test_base_1
Output Folder : ../output_file/MODEL_V1_NYC/
Tfgraph Folder : ../tfgraph/MODEL_V1_NYC/0827_NY_001_2ch_test_base_1
Save Folder : ../saved_model/MODEL_V1_NYC/


In [2]:
########################################
## Data Load
########################################


NYC_FOLDER = '/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/'

x_st_train = np.log(1.0+load_np_data(NYC_FOLDER+'train_st_x_2d.npz'))
x_ed_train = np.log(1.0+load_np_data(NYC_FOLDER+'train_end_x_2d.npz'))
y_train = np.log(1.0+load_np_data(NYC_FOLDER+'train_st_y_2d.npz'))
temporal_train = load_np_data(NYC_FOLDER+'temporal_train.npz')


x_st_test = np.log(1.0+load_np_data(NYC_FOLDER+'test_st_x_2d.npz'))
x_ed_test = np.log(1.0+load_np_data(NYC_FOLDER+'test_end_x_2d.npz'))
y_test = np.log(1.0+load_np_data(NYC_FOLDER+'test_st_y_2d.npz'))
temporal_test = load_np_data(NYC_FOLDER+'temporal_test.npz')

coord_train = load_np_data(NYC_FOLDER+'train_coord_x_2d.npz')/10 #args.coord
coord_test = load_np_data(NYC_FOLDER+'test_coord_x_2d.npz')/10 #args.coord

x_st_train = x_st_train[:-1]
x_ed_train = x_ed_train[:-1]

x_st_test = x_st_test[:-1]
x_ed_test = x_ed_test[:-1]

y_t2_train = y_train[1:]
y_t2_test = y_test[1:]

y_train = y_train[:-1]
y_test = y_test[:-1]

x_t2_temporal_train = temporal_train[1:]
x_t2_temporal_test = temporal_test[1:]

x_t1_temporal_train = temporal_train[:-1]
x_t1_temporal_test = temporal_test[:-1]

x_st_train = np.concatenate((x_st_train,coord_train[:-1]), axis=3)
x_st_test = np.concatenate((x_st_test,coord_test[:-1]), axis=3)

#y_train = np.concatenate( (y_train, y_t2_train), axis=3)
#y_test = np.concatenate( (y_test, y_t2_test), axis=3)

print (x_st_train.shape, x_ed_train.shape, y_train.shape, x_t1_temporal_train.shape,x_t2_temporal_train.shape, y_t2_train.shape)
print (x_st_test.shape, x_ed_test.shape, y_test.shape, x_t1_temporal_test.shape,x_t2_temporal_test.shape, y_t2_test.shape)


/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/train_st_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/train_end_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/train_st_y_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/temporal_train.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/test_st_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/test_end_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/test_st_y_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/temporal_test.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/train_coord_x_2d.npz Loaded
/data/public/rw/prj-mobility/10.195.12.143/kakaobrain/taxi/datasets/NY_data/test_coord_x

## Data verification

In [3]:
print (np.average(np.equal(x_st_train[1,:,:,-3], y_train[0,:,:,0])))
print (np.average(np.equal(x_st_train[0,:,:,-3], x_st_train[1,:,:,-4])))

1.0
1.0


In [4]:
## Make Validation Data About 20% 
val_idx = 1524

print ('----- Before Split ----')
#print (x_st_train.shape, x_ed_train.shape, x_t1_temporal_train.shape, x_t0_temporal_train.shape, x_t2_temporal_train.shape)

x_st_val = x_st_train[val_idx:]
x_ed_val = x_ed_train[val_idx:]
y_val = y_train[val_idx:]
x_t1_temporal_val = x_t1_temporal_train[val_idx:]
#x_t0_temporal_val = x_t0_temporal_train[val_idx:]
x_t2_temporal_val = x_t2_temporal_train[val_idx:]


x_st_train = x_st_train[:val_idx]
x_ed_train = x_ed_train[:val_idx]
y_train = y_train[:val_idx]
x_t1_temporal_train = x_t1_temporal_train[:val_idx]
#x_t0_temporal_train = x_t0_temporal_train[:val_idx]
x_t2_temporal_train = x_t2_temporal_train[:val_idx]


print ('----- After Split ----')
print (x_st_train.shape, x_ed_train.shape, x_t1_temporal_train.shape, x_t2_temporal_train.shape)
print (x_st_val.shape, x_ed_val.shape, x_t1_temporal_val.shape, x_t2_temporal_val.shape)


----- Before Split ----
----- After Split ----
(1524, 10, 20, 10) (1524, 10, 20, 16) (1524, 10, 20, 57) (1524, 10, 20, 57)
(379, 10, 20, 10) (379, 10, 20, 16) (379, 10, 20, 57) (379, 10, 20, 57)


In [5]:
K.clear_session()
print ('Session Reseted')

model_input_train_data = [x_st_train, x_ed_train, x_t1_temporal_train, x_t2_temporal_train]
model_input_val_data=[x_st_val, x_ed_val, x_t1_temporal_val, x_t2_temporal_val]
model_input_test_data = [x_st_test, x_ed_test, x_t1_temporal_test, x_t2_temporal_test]


def make_temporal_model(st_data, ed_data, tmp_data):

    im_h,im_w,im_c = st_data[0].shape

    start_input = Input(shape=(st_data[0].shape))
    end_input = Input(shape=(ed_data[0].shape))
    t1_temporal_input = Input(shape=(tmp_data[0].shape))
    t2_temporal_input = Input(shape=(tmp_data[0].shape))

    input_tensors = [start_input, end_input, t1_temporal_input, t2_temporal_input]

    coord_input = keras.layers.Lambda(lambda xin: xin[:,:,:,-2:] )(start_input)
    start_input = keras.layers.Lambda(lambda xin: xin[:,:,:,:-2] )(start_input)

    
    net_t1_temp = Conv2D(16, kernel_size=(1,1), strides=(1,1))(t1_temporal_input)
    net_t1_temp = Conv2D(16, kernel_size=(1,1), strides=(1,1))(net_t1_temp)
    
    y = layers.concatenate([start_input, net_t1_temp], axis=-1)
    
    y = Conv2D(128, kernel_size=(5, 5), activation='relu', padding='same')(y)
    y = BatchNormalization()(y)
    y = Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same')(y)
    y = BatchNormalization()(y)
    y = keras.layers.GaussianNoise(0.5)(y)
    y = Conv2D(32, kernel_size=(5, 5), activation='relu', padding='same')(y)
    y = BatchNormalization()(y)
    y = keras.layers.GaussianNoise(0.5)(y)
    y = Conv2D(16, kernel_size=(5, 5), activation='relu', padding='same')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.5)(y)
    
    y2 = Conv2D(8, kernel_size=(5, 5), activation='relu', padding='same')(end_input)
    y2 = BatchNormalization()(y2)
    y2 = keras.layers.GaussianNoise(0.5)(y2)
    y2 = Conv2D(8, kernel_size=(5, 5), activation='relu', padding='same')(y2)
    y2 = BatchNormalization()(y2)
    
    y = layers.concatenate([y, net_t1_temp, y2, start_input, end_input, coord_input], axis=-1)
    
    y = Conv2D(256, kernel_size=(5, 5), activation='relu', padding='same')(y)
    y = BatchNormalization()(y)
    y = keras.layers.GaussianNoise(0.5)(y)
    y = Conv2D(128, kernel_size=(5, 5), activation='relu', padding='same')(y)
    y = BatchNormalization()(y)
    y = Conv2D(64, kernel_size=(5, 5), activation='relu', padding='same')(y)
    y = BatchNormalization()(y)
    y = Dropout(0.5)(y)
    output = Conv2D(1, kernel_size=(5, 5), activation='relu', padding='same')(y)
    

    model = Model(inputs=input_tensors, outputs=output)
    return model



b_model = make_temporal_model(x_st_train,x_ed_train, x_t1_temporal_train)
model = multi_gpu_model(b_model, gpus=NGPU)
print (MODEL_NAME_, 'Model Created')
print (b_model.summary())


Session Reseted
MODEL_V1_NYC_0827_NY_001_2ch_test_base_1 Model Created
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 10, 20, 57)   0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 10, 20, 10)   0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 10, 20, 16)   928         input_3[0][0]                    
__________________________________________________________________________________________________
lambda_2 (Lambda)               (None, 10, 20, 8)    0           input_1[0][0]                    
______________________________________

In [6]:
########################################
## Callback 
########################################


from keras.callbacks import ModelCheckpoint
from keras.optimizers import SGD, Adam

from keras.callbacks import Callback

class SGDLearningRateTracker(Callback):
    def on_epoch_end(self, epoch, logs={}):
        optimizer = self.model.optimizer
        lr = K.eval(optimizer.lr) # K.eval(optimizer.lr * (1. / (1. + optimizer.decay * optimizer.iterations)))
        print('LR: {:.6f}'.format(lr))

tb_hist = keras.callbacks.TensorBoard(log_dir=TF_FOLDER, histogram_freq=0, write_graph=True, write_images=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=50, verbose=0, mode='min')



## Training 

#### MSE pretraining -> MAE fine-tuning 


In [7]:

import time
start_time = time.time() 


#model.compile(loss=['mean_squared_error'], optimizer=Adam(lr=LRATE,  decay=0.01), metrics=['mean_absolute_error', invlog_mape_tr10])
#history = model.fit(model_input_train_data , y_train,
#                  batch_size=BATCH_SIZE,
#                  epochs=EPOCHS,
#                  callbacks=[tb_hist, SGDLearningRateTracker()],
#                  shuffle = True,
#                  verbose=2
#                  ,validation_data=(model_input_val_data, y_val)
#                 )
    
#val_loss = history.history['val_loss']
#val_metric = history.history['val_mean_absolute_error']

#save_model_name = MODEL_SAVE_FOLDER+MODEL_NAME_+'st1.h5'
#b_model.save(save_model_name)
#print ('Model Saved', save_model_name)
#history_save(val_loss, val_metric, model_name =  MODEL_NAME_+'st1', output_folder=OUTPUT_FOLDER)


model.compile(loss=['mean_absolute_error'], optimizer=Adam(lr=LRATE,  decay=0.01), metrics=['mean_absolute_error', invlog_mape_tr10])
history = model.fit(model_input_train_data , y_train,
                  batch_size=BATCH_SIZE,
                  epochs=EPOCHS,
                  callbacks=[tb_hist, early_stopping, SGDLearningRateTracker()],
                  shuffle = True,
                  verbose=2
                  ,validation_data=(model_input_val_data, y_val)
                 )

val_loss2 = history.history['val_loss']
val_metric2 = history.history['val_mean_absolute_error']

end_time = time.time()
n_epochs = len(val_metric2) #len(val_metric)+len(val_metric2)

print ('')
print("--- Train Time : %0.2f hour  ---" %(  (end_time - start_time)/3600  ))
print("--- # of Epochs: %0.f  ---" %( n_epochs ) )


save_model_name = MODEL_SAVE_FOLDER+MODEL_NAME_+'st2.h5'
b_model.save(save_model_name)
print ('Model Saved', save_model_name)
history_save(val_loss2, val_metric2, model_name =  MODEL_NAME_+'st2', output_folder=OUTPUT_FOLDER)


Train on 1524 samples, validate on 379 samples
Epoch 1/2000
 - 7s - loss: 0.7684 - mean_absolute_error: 0.7684 - invlog_mape_tr10: 1669.4385 - val_loss: 0.5935 - val_mean_absolute_error: 0.5935 - val_invlog_mape_tr10: 5.0158
LR: 0.001000
Epoch 2/2000
 - 1s - loss: 0.4890 - mean_absolute_error: 0.4890 - invlog_mape_tr10: 2.0178 - val_loss: 0.4380 - val_mean_absolute_error: 0.4380 - val_invlog_mape_tr10: 2.1322
LR: 0.001000
Epoch 3/2000
 - 1s - loss: 0.4190 - mean_absolute_error: 0.4190 - invlog_mape_tr10: 1.2826 - val_loss: 0.3359 - val_mean_absolute_error: 0.3359 - val_invlog_mape_tr10: 0.9860
LR: 0.001000
Epoch 4/2000
 - 1s - loss: 0.3637 - mean_absolute_error: 0.3637 - invlog_mape_tr10: 0.9164 - val_loss: 0.3292 - val_mean_absolute_error: 0.3292 - val_invlog_mape_tr10: 0.7893
LR: 0.001000
Epoch 5/2000
 - 1s - loss: 0.3251 - mean_absolute_error: 0.3251 - invlog_mape_tr10: 0.7285 - val_loss: 0.4560 - val_mean_absolute_error: 0.4560 - val_invlog_mape_tr10: 1.3499
LR: 0.001000
Epoch 6/20

LR: 0.001000
Epoch 45/2000
 - 1s - loss: 0.2098 - mean_absolute_error: 0.2098 - invlog_mape_tr10: 0.3377 - val_loss: 0.1865 - val_mean_absolute_error: 0.1865 - val_invlog_mape_tr10: 0.2756
LR: 0.001000
Epoch 46/2000
 - 1s - loss: 0.2078 - mean_absolute_error: 0.2078 - invlog_mape_tr10: 0.3405 - val_loss: 0.1728 - val_mean_absolute_error: 0.1728 - val_invlog_mape_tr10: 0.2085
LR: 0.001000
Epoch 47/2000
 - 1s - loss: 0.2060 - mean_absolute_error: 0.2060 - invlog_mape_tr10: 0.3293 - val_loss: 0.1768 - val_mean_absolute_error: 0.1768 - val_invlog_mape_tr10: 0.2033
LR: 0.001000
Epoch 48/2000
 - 1s - loss: 0.2063 - mean_absolute_error: 0.2063 - invlog_mape_tr10: 0.3367 - val_loss: 0.1708 - val_mean_absolute_error: 0.1708 - val_invlog_mape_tr10: 0.1873
LR: 0.001000
Epoch 49/2000
 - 1s - loss: 0.2074 - mean_absolute_error: 0.2074 - invlog_mape_tr10: 0.3281 - val_loss: 0.1745 - val_mean_absolute_error: 0.1745 - val_invlog_mape_tr10: 0.1999
LR: 0.001000
Epoch 50/2000
 - 1s - loss: 0.2064 - mean_

LR: 0.001000
Epoch 89/2000
 - 1s - loss: 0.1934 - mean_absolute_error: 0.1934 - invlog_mape_tr10: 0.2910 - val_loss: 0.1706 - val_mean_absolute_error: 0.1706 - val_invlog_mape_tr10: 0.1877
LR: 0.001000
Epoch 90/2000
 - 1s - loss: 0.1948 - mean_absolute_error: 0.1948 - invlog_mape_tr10: 0.2972 - val_loss: 0.1687 - val_mean_absolute_error: 0.1687 - val_invlog_mape_tr10: 0.1826
LR: 0.001000
Epoch 91/2000
 - 1s - loss: 0.1941 - mean_absolute_error: 0.1941 - invlog_mape_tr10: 0.2924 - val_loss: 0.1685 - val_mean_absolute_error: 0.1685 - val_invlog_mape_tr10: 0.1831
LR: 0.001000
Epoch 92/2000
 - 1s - loss: 0.1941 - mean_absolute_error: 0.1941 - invlog_mape_tr10: 0.2954 - val_loss: 0.1727 - val_mean_absolute_error: 0.1727 - val_invlog_mape_tr10: 0.1920
LR: 0.001000
Epoch 93/2000
 - 1s - loss: 0.1939 - mean_absolute_error: 0.1939 - invlog_mape_tr10: 0.2946 - val_loss: 0.1726 - val_mean_absolute_error: 0.1726 - val_invlog_mape_tr10: 0.1930
LR: 0.001000
Epoch 94/2000
 - 1s - loss: 0.1935 - mean_

LR: 0.001000
Epoch 133/2000
 - 1s - loss: 0.1898 - mean_absolute_error: 0.1898 - invlog_mape_tr10: 0.2858 - val_loss: 0.1673 - val_mean_absolute_error: 0.1673 - val_invlog_mape_tr10: 0.1792
LR: 0.001000
Epoch 134/2000
 - 1s - loss: 0.1891 - mean_absolute_error: 0.1891 - invlog_mape_tr10: 0.2795 - val_loss: 0.1648 - val_mean_absolute_error: 0.1648 - val_invlog_mape_tr10: 0.1792
LR: 0.001000
Epoch 135/2000
 - 1s - loss: 0.1896 - mean_absolute_error: 0.1896 - invlog_mape_tr10: 0.2826 - val_loss: 0.1654 - val_mean_absolute_error: 0.1654 - val_invlog_mape_tr10: 0.1781
LR: 0.001000
Epoch 136/2000
 - 1s - loss: 0.1896 - mean_absolute_error: 0.1896 - invlog_mape_tr10: 0.2836 - val_loss: 0.1639 - val_mean_absolute_error: 0.1639 - val_invlog_mape_tr10: 0.1727
LR: 0.001000
Epoch 137/2000
 - 1s - loss: 0.1882 - mean_absolute_error: 0.1882 - invlog_mape_tr10: 0.2780 - val_loss: 0.1660 - val_mean_absolute_error: 0.1660 - val_invlog_mape_tr10: 0.1766
LR: 0.001000
Epoch 138/2000
 - 1s - loss: 0.1895 -

 - 1s - loss: 0.1861 - mean_absolute_error: 0.1861 - invlog_mape_tr10: 0.2735 - val_loss: 0.1637 - val_mean_absolute_error: 0.1637 - val_invlog_mape_tr10: 0.1736
LR: 0.001000
Epoch 177/2000
 - 1s - loss: 0.1865 - mean_absolute_error: 0.1865 - invlog_mape_tr10: 0.2794 - val_loss: 0.1782 - val_mean_absolute_error: 0.1782 - val_invlog_mape_tr10: 0.2072
LR: 0.001000
Epoch 178/2000
 - 1s - loss: 0.1859 - mean_absolute_error: 0.1859 - invlog_mape_tr10: 0.2714 - val_loss: 0.1727 - val_mean_absolute_error: 0.1727 - val_invlog_mape_tr10: 0.1928
LR: 0.001000
Epoch 179/2000
 - 1s - loss: 0.1851 - mean_absolute_error: 0.1851 - invlog_mape_tr10: 0.2697 - val_loss: 0.1649 - val_mean_absolute_error: 0.1649 - val_invlog_mape_tr10: 0.1732
LR: 0.001000
Epoch 180/2000
 - 1s - loss: 0.1853 - mean_absolute_error: 0.1853 - invlog_mape_tr10: 0.2714 - val_loss: 0.1725 - val_mean_absolute_error: 0.1725 - val_invlog_mape_tr10: 0.1927
LR: 0.001000
Epoch 181/2000
 - 1s - loss: 0.1852 - mean_absolute_error: 0.1852

...

LR: 0.001000
Epoch 742/2000
 - 1s - loss: 0.1702 - mean_absolute_error: 0.1702 - invlog_mape_tr10: 0.2401 - val_loss: 0.1641 - val_mean_absolute_error: 0.1641 - val_invlog_mape_tr10: 0.1675
LR: 0.001000
Epoch 743/2000
 - 1s - loss: 0.1705 - mean_absolute_error: 0.1705 - invlog_mape_tr10: 0.2424 - val_loss: 0.1633 - val_mean_absolute_error: 0.1633 - val_invlog_mape_tr10: 0.1659
LR: 0.001000
Epoch 744/2000
 - 1s - loss: 0.1704 - mean_absolute_error: 0.1704 - invlog_mape_tr10: 0.2393 - val_loss: 0.1614 - val_mean_absolute_error: 0.1614 - val_invlog_mape_tr10: 0.1627
LR: 0.001000
Epoch 745/2000
 - 1s - loss: 0.1707 - mean_absolute_error: 0.1707 - invlog_mape_tr10: 0.2414 - val_loss: 0.1627 - val_mean_absolute_error: 0.1627 - val_invlog_mape_tr10: 0.1643
LR: 0.001000
Epoch 746/2000
 - 1s - loss: 0.1703 - mean_absolute_error: 0.1703 - invlog_mape_tr10: 0.2391 - val_loss: 0.1629 - val_mean_absolute_error: 0.1629 - val_invlog_mape_tr10: 0.1646
LR: 0.001000
Epoch 747/2000
 - 1s - loss: 0.1703 -

 - 1s - loss: 0.1700 - mean_absolute_error: 0.1700 - invlog_mape_tr10: 0.2400 - val_loss: 0.1622 - val_mean_absolute_error: 0.1622 - val_invlog_mape_tr10: 0.1644
LR: 0.001000
Epoch 786/2000
 - 1s - loss: 0.1696 - mean_absolute_error: 0.1696 - invlog_mape_tr10: 0.2392 - val_loss: 0.1625 - val_mean_absolute_error: 0.1625 - val_invlog_mape_tr10: 0.1635
LR: 0.001000
Epoch 787/2000
 - 1s - loss: 0.1700 - mean_absolute_error: 0.1700 - invlog_mape_tr10: 0.2399 - val_loss: 0.1627 - val_mean_absolute_error: 0.1627 - val_invlog_mape_tr10: 0.1648
LR: 0.001000
Epoch 788/2000
 - 1s - loss: 0.1698 - mean_absolute_error: 0.1698 - invlog_mape_tr10: 0.2409 - val_loss: 0.1640 - val_mean_absolute_error: 0.1640 - val_invlog_mape_tr10: 0.1673
LR: 0.001000
Epoch 789/2000
 - 1s - loss: 0.1699 - mean_absolute_error: 0.1699 - invlog_mape_tr10: 0.2410 - val_loss: 0.1635 - val_mean_absolute_error: 0.1635 - val_invlog_mape_tr10: 0.1659
LR: 0.001000
Epoch 790/2000
 - 1s - loss: 0.1700 - mean_absolute_error: 0.1700

LR: 0.001000
Epoch 829/2000
 - 1s - loss: 0.1694 - mean_absolute_error: 0.1694 - invlog_mape_tr10: 0.2399 - val_loss: 0.1629 - val_mean_absolute_error: 0.1629 - val_invlog_mape_tr10: 0.1643
LR: 0.001000
Epoch 830/2000
 - 1s - loss: 0.1695 - mean_absolute_error: 0.1695 - invlog_mape_tr10: 0.2397 - val_loss: 0.1632 - val_mean_absolute_error: 0.1632 - val_invlog_mape_tr10: 0.1648
LR: 0.001000
Epoch 831/2000
 - 1s - loss: 0.1697 - mean_absolute_error: 0.1697 - invlog_mape_tr10: 0.2362 - val_loss: 0.1611 - val_mean_absolute_error: 0.1611 - val_invlog_mape_tr10: 0.1634
LR: 0.001000
Epoch 832/2000
 - 1s - loss: 0.1694 - mean_absolute_error: 0.1694 - invlog_mape_tr10: 0.2399 - val_loss: 0.1610 - val_mean_absolute_error: 0.1610 - val_invlog_mape_tr10: 0.1619
LR: 0.001000
Epoch 833/2000
 - 1s - loss: 0.1689 - mean_absolute_error: 0.1689 - invlog_mape_tr10: 0.2395 - val_loss: 0.1632 - val_mean_absolute_error: 0.1632 - val_invlog_mape_tr10: 0.1652
LR: 0.001000
Epoch 834/2000
 - 1s - loss: 0.1696 -

 - 1s - loss: 0.1691 - mean_absolute_error: 0.1691 - invlog_mape_tr10: 0.2386 - val_loss: 0.1645 - val_mean_absolute_error: 0.1645 - val_invlog_mape_tr10: 0.1673
LR: 0.001000
Epoch 873/2000
 - 1s - loss: 0.1689 - mean_absolute_error: 0.1689 - invlog_mape_tr10: 0.2378 - val_loss: 0.1629 - val_mean_absolute_error: 0.1629 - val_invlog_mape_tr10: 0.1640
LR: 0.001000
Epoch 874/2000
 - 1s - loss: 0.1684 - mean_absolute_error: 0.1684 - invlog_mape_tr10: 0.2356 - val_loss: 0.1640 - val_mean_absolute_error: 0.1640 - val_invlog_mape_tr10: 0.1667
LR: 0.001000
Epoch 875/2000
 - 1s - loss: 0.1683 - mean_absolute_error: 0.1683 - invlog_mape_tr10: 0.2363 - val_loss: 0.1635 - val_mean_absolute_error: 0.1635 - val_invlog_mape_tr10: 0.1658
LR: 0.001000
Epoch 876/2000
 - 1s - loss: 0.1686 - mean_absolute_error: 0.1686 - invlog_mape_tr10: 0.2370 - val_loss: 0.1620 - val_mean_absolute_error: 0.1620 - val_invlog_mape_tr10: 0.1623
LR: 0.001000
Epoch 877/2000
 - 1s - loss: 0.1686 - mean_absolute_error: 0.1686

LR: 0.001000
Epoch 916/2000
 - 1s - loss: 0.1685 - mean_absolute_error: 0.1685 - invlog_mape_tr10: 0.2383 - val_loss: 0.1631 - val_mean_absolute_error: 0.1631 - val_invlog_mape_tr10: 0.1648
LR: 0.001000
Epoch 917/2000
 - 1s - loss: 0.1686 - mean_absolute_error: 0.1686 - invlog_mape_tr10: 0.2366 - val_loss: 0.1614 - val_mean_absolute_error: 0.1614 - val_invlog_mape_tr10: 0.1618
LR: 0.001000
Epoch 918/2000
 - 1s - loss: 0.1686 - mean_absolute_error: 0.1686 - invlog_mape_tr10: 0.2377 - val_loss: 0.1613 - val_mean_absolute_error: 0.1613 - val_invlog_mape_tr10: 0.1618
LR: 0.001000
Epoch 919/2000
 - 1s - loss: 0.1685 - mean_absolute_error: 0.1685 - invlog_mape_tr10: 0.2383 - val_loss: 0.1625 - val_mean_absolute_error: 0.1625 - val_invlog_mape_tr10: 0.1633
LR: 0.001000
Epoch 920/2000
 - 1s - loss: 0.1687 - mean_absolute_error: 0.1687 - invlog_mape_tr10: 0.2386 - val_loss: 0.1627 - val_mean_absolute_error: 0.1627 - val_invlog_mape_tr10: 0.1635
LR: 0.001000

--- Train Time : 0.25 hour  ---
--- 

## Model Output Print

In [8]:
pred_test = model.predict(model_input_test_data)

print ('')
print ("## Test datasets Performance")
print ("- MAPE(11 or more) : %.3f"%mape_trs(inverse_logscale(y_test), inverse_logscale(pred_test), 11))
print ("- RMSE(11 or more) : %.3f"%rmse_trs(inverse_logscale(y_test), inverse_logscale(pred_test), 11))
print ('')

#print ("- T1 MAPE(11 or more) : %.3f"%mape_trs(inverse_logscale(y_test[:,:,:,:1]), inverse_logscale(pred_test[:,:,:,:1]), 11))
#print ("- T2 MAPE(11 or more) : %.3f"%mape_trs(inverse_logscale(y_test[:,:,:,-1:]), inverse_logscale(pred_test[:,:,:,-1:]), 11))




## Test datasets Performance
- MAPE(11 or more) : 0.164
- RMSE(11 or more) : 31.304



In [10]:

history_save(val_loss2, val_metric2, model_name =  MODEL_NAME_, output_folder=OUTPUT_FOLDER)
make_test_ouput(model, model_input_test_data , y_test, model_name=MODEL_NAME_, norm=SCALE, output_folder=OUTPUT_FOLDER)
#make_test_2ch_ouput(model, model_input_test_data , y_test, model_name=MODEL_NAME_, norm=SCALE, output_folder=OUTPUT_FOLDER)



../output_file/MODEL_V1_NYC/MODEL_V1_NYC_0827_NY_001_2ch_test_base_1_loss_metric_invmape_0.163.csv  Val loss Saved
Test True saved :  ../output_file/MODEL_V1_NYC/MODEL_V1_NYC_0827_NY_001_2ch_test_base_1_gt.csv
Test Pred saved :  ../output_file/MODEL_V1_NYC/MODEL_V1_NYC_0827_NY_001_2ch_test_base_1_pred.csv



In [11]:

save_model_name = MODEL_SAVE_FOLDER+MODEL_NAME_+'.h5'
b_model.save(save_model_name)
print ('Model Saved', save_model_name)


Model Saved MODEL_V1_NYC_0827_NY_001_2ch_test_base_1MODEL_V1_NYC_0827_NY_001_2ch_test_base_1.h5
